In [ ]:
from utils import network
from utils import visualization
from utils import tissue
from utils import func
import copy
import numpy as np
import pandas as pd
import pylab as plt
import scipy
import seaborn as sns

#### Section 1.  Load dataset and instantiate a multiplex network object

In [ ]:
%cd data
curated_cell_label = "Classical monocytes"
tissue_ns = ['BLD', 'BMA','LIV', 'LNG', 'SPL']
for name in tissue_ns:
    globals()[name] = np.load(f'{name}_{curated_cell_label}_GO_CC_immunesig.npy', allow_pickle = True).tolist()

Sanity check

In [ ]:
### The filtered gene sets are the same 
collection = "immunesig"
method = 'kmeans'
length = len(tissue_ns)
filter_check = np.array([eval(i).gene_set[collection].filter_index[method] \
                         == eval(j).gene_set[collection].filter_index[method] \
                         for i in tissue_ns for j in tissue_ns]).reshape(length, length)
if np.all(filter_check):
    print(f'The filtered gene sets of {collection} are the same in each layer')

In [ ]:
from imp import reload
reload(network)

In [ ]:
multiplex_nw_classical_monocyte = network.multiplex_tissue_network(gene_set_collection_name = collection, layer = [eval(tis) for tis in tissue_ns],\
                                                layer_ns = tissue_ns, method = method, self_loop = True)
multiplex_nw_classical_monocyte.load_Jaccard(self_loop = True)

In [ ]:
### Remove edges with negative or low weights
multiplex_nw_classical_monocyte.call_adj_filter_tr(q = None, tr = 0.2)

$~$

#### Section 2: Analysis on the structures of the multiplex network

#### Community detection

In [ ]:
### Modularity Vertex Partition, mininal size of a cluster is set to be 50.
multiplex_nw_classical_monocyte.community_detection_MVP(min_cluster = 50, MVP_para = {'weights': "weight", 'n_iterations':-1, 'seed':123})

In [ ]:
membership_classical_monocyte = multiplex_nw_classical_monocyte.community_detection['MVP'][0] 
membership_classical_monocyte = np.array(membership_classical_monocyte)
np.unique(membership_classical_monocyte, return_counts = True)

$~$

#### Graph embedding by UMAP featured by community memberships or centrality coefficients

In [ ]:
for multiple_nw in [multiplex_nw_classical_monocyte, ]:
    multiple_nw.centrality_per_layer(method = "hub")

In [ ]:
fig_lst, vis_lst = visualization.vis_multiplex_network(multiplex_nw_classical_monocyte, display_feature = 'membership', title_size = 22)

In [ ]:
np.save('figures/vis_multiplex_UMAP_coord.npy', vis_lst)

layer = 0
for fig in fig_lst :
    fig.savefig(f"figures/UMAPS/{layer}_UMAP_membership.png",  dpi = 300)
    layer += 1

In [ ]:
fig_lst, vis_lst = visualization.vis_multiplex_network(multiplex_nw_classical_monocyte, UMAP_coord = vis_lst, title_size = 22, display_feature = 'hub')

$~$

In [ ]:
layer = 0
for fig in fig_lst :
    fig.savefig(f"figures/UMAPS/{layer}_UMAP_centrality.png",  dpi = 300)
    layer += 1

$~$

**Layer similarity** 

Reference: Battiston, Federico, Vincenzo Nicosia, and Vito Latora. "The new challenges of multiplex networks: Measures and models." The European Physical Journal Special Topics 226 (2017)

In [ ]:
tissue_ns.insert(0, 'Jaccard')

In [ ]:
multiplex_nw_classical_monocyte.community_detection_MVP_per_layer(MVP_para = {'weights': "weight", 'n_iterations':-1, 'seed':123}, Jaccard = True)
multiplex_nw_classical_monocyte.NMI_layers()
layer_similarity = multiplex_nw_classical_monocyte.multiplex_property['inter-layer similarity']

$~$

In [ ]:
fig = visualization.vis_heatmap(layer_similarity, tissue_ns, ticks = np.arange(0.5,len(tissue_ns)+0.5,1),\
                                x_rotation = 20, y_rotation = 20, anno_font_size = 20, cbar_font = 18, title_size = 22, figsize=(12,12),\
                                title = "Layers in the multiplex network", \
                                para_heatmap = {'annot':True, 'vmin':0, 'vmax':1, 'cmap':'magma',\
                                                'square':True, 'cbar_kws': {"shrink": .15, "aspect":8, "anchor" :(0.0, 0.85)}}
                               )

In [ ]:
fig.savefig(f'figures/NMI_layers_{collection}_{method}.png', dpi = 300)

$~$

**Structural coefficients** 

In [ ]:
multiplex_nw_classical_monocyte.multiplex_participation_coefficient()
O = multiplex_nw_classical_monocyte.multiplex_property['O']
P = multiplex_nw_classical_monocyte.multiplex_property['P']

In [ ]:
np.save('structural_property/O.npy', O)
np.save('structural_property/P.npy', P)

In [ ]:
O = np.load('structural_property/O.npy')
P = np.load('structural_property/P.npy')

$~$

In [ ]:
fig = visualization.vis_scatter(O, P, "Total weight (O)", "Multiplex participation coefficient", para_jointplot = {'color':'g', 'alpha':0.2, 'kind':'scatter', 'space':0.9}, \
                                              xtick_integer = True, anno_font_size = 18, height = 8, ratio = 5)

In [ ]:
fig.savefig('figures/O_P_plot.png', dpi = 300)

In [ ]:
multiplex_nw_classical_monocyte.C_1()
C1 = multiplex_nw_classical_monocyte.multiplex_property['C1']

In [ ]:
multiplex_nw_classical_monocyte.C_2()
C2 = multiplex_nw_classical_monocyte.multiplex_property['C2']

In [ ]:
np.save('structural_property/C1.npy', C1)
np.save('structural_property/C2.npy', C2)

In [ ]:
C1 = np.load('structural_property/C1.npy')
C2 = np.load('structural_property/C2.npy')

$~$

In [ ]:
fig = visualization.vis_scatter(C1, C2, "C1", "C2", \
                                para_jointplot = {'color':'y', 'alpha':0.25,\
                                                  'xlim':(0.0,0.6), 'ylim':(0.0, 0.6), 'kind':'scatter', 'space':0.9},\
                               anno_font_size = 18, height = 8, ratio = 8)

In [ ]:
fig.savefig('figures/C1_C2.png', dpi = 300)

In [ ]:
multiplex_nw_classical_monocyte.multiplex_page_rank()
multiplex_nw_classical_monocyte.Jaccard_page_rank()

In [ ]:
mpg = multiplex_nw_classical_monocyte.multiplex_property['multiplex_page_rank']
jpg = multiplex_nw_classical_monocyte.multiplex_property['Jaccard_page_rank']

In [ ]:
np.save('structural_property/mpg.npy', mpg)
np.save('structural_property/jpg.npy', jpg)

In [ ]:
mpg = np.load('structural_property/mpg.npy')
jpg = np.load('structural_property/jpg.npy')

$~$

In [ ]:
fig = visualization.vis_scatter(jpg/(1/(len(jpg))), mpg/(1/(len(mpg))),  "PageRank (ratio) in the Jaccard layer", "Multiplex PageRank (ratio)",\
                          para_jointplot = {'color':'r', 'alpha':0.3, 'xlim':(0, 2),\
                                            'ylim' : (0,30), 'kind':'scatter', 'space':0.9, 'marginal_kws':dict(bins=100)},\
                               anno_font_size = 18, height = 8, ratio = 5)

In [ ]:
fig.savefig('figures/centrality.png', dpi = 300)

$~$

In [ ]:
fig = visualization.vis_scatter(gene_set_size, mpg/(1/(len(mpg))),  "Gene set size", "Multiplex PageRank (ratio)",\
                          xtick_integer = True, para_jointplot = {'color':'b', 'alpha':0.3, 'xlim':(np.min(gene_set_size)-5,np.max(gene_set_size)+2),\
                                            'ylim' : (0,30), 'kind':'scatter', 'space':0.9, 'marginal_kws':dict(bins=100)},\
                               anno_font_size = 18, height = 8, ratio = 5)

In [ ]:
fig.savefig('figures/gene_set_size_centrality.png', dpi = 300)

**Correlations among gene set attributes**

In [ ]:
# tissue_ns.pop(0)

In [ ]:
filter_index = SPL.gene_set[collection].filter_index['kmeans']
gene_set_size = np.array([len(item) for item in SPL.gene_set[collection].gene_set])[filter_index]
items = [O, P, C1, C2, mpg/(1/len(mpg)), jpg/(1/len(jpg)), gene_set_size, membership_classical_monocyte]   
items = [np.round(item, 2) for item in items]
labels = ['Total weight', 'P', 'C1', 'C2', 'Multiplex PageRank', 'PageRank-Jaccard', 'gene set size', 'multiplex community']

data = pd.DataFrame(data = np.array(items).transpose(), columns = labels)
data['multiplex community'] = data['multiplex community'].astype('int')+1

In [ ]:
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.figure(figsize=(19,19))
plt.rc('legend',fontsize=25, title_fontsize=25)
plt.rc('axes', labelsize = 18 )
plt.rc('xtick', labelsize = 16)
plt.rc('ytick', labelsize = 16)
fig = sns.pairplot(data, corner = True, hue = 'multiplex community', kind = 'scatter', palette = "tab10", height = 5.5)
plt.setp(fig._legend.get_texts(), fontsize=30) 
plt.setp(fig._legend.get_title(), fontsize=34)

In [ ]:
fig.figure.savefig('figures/attribute_pair_plot.png', dpi = 150)

In [ ]:
items.pop(-1)
labels.pop(-1)

In [ ]:
### correlation
cor_res = np.array([scipy.stats.spearmanr(i,j)[0] for i in items for j in items]).reshape(len(items),len(items))
# ### p values
# cor_p = np.array([scipy.stats.spearmanr(i,j)[1] for i in items for j in items]).reshape(len(items),len(items))

$~$

In [ ]:
fig = visualization.vis_heatmap(cor_res, x_rotation = 20, y_rotation = 20, \
                                figsize=(15,15), labels = labels, \
                                legend_title = "", cbar_font = 18,
                                anno_font_size = 20, title_size = 22,\
                                title = "Spearman correlations among attributes of gene sets", ticks = np.arange(0.5,len(items)+0.5,1),\
                               para_heatmap = {'annot':True, 'vmin':-1, 'vmax':1, \
                                               'cmap':'magma', 'square':True,\
                                               'cbar_kws': {"shrink": .15, "aspect":8, "anchor" :(0.0, 0.85), "ticks":[-1 + 0.5*i for i in range(5)]}})

In [ ]:
fig.savefig('figures/cor_plot.png', dpi = 300)

$~$

#### Appendix:  Transfer to R for Grimon 

In [ ]:
### For visualization in Grimon (R package)
M = len(vis_lst)
vis_df = np.concatenate([vis_lst[i].drop(["feature"], axis = 1).to_numpy() for i in range(M)], axis = 1)
### Featured by the community membership
palette = list(sns.color_palette(n_colors = len(np.unique(membership_classical_monocyte))).as_hex())
color_membership = [palette[select] for select in membership_classical_monocyte]

In [ ]:
color_high_mpg = ['#ff7f0e' if top else '#7f7f7f' for top in mpg >= np.quantile(mpg, 0.9)]
color_high_C2 = ['#17becf' if top else '#7f7f7f' for top in C2 >= np.quantile(C2, 0.9)]
color_low_P = ['#e377c2' if top else '#7f7f7f' for top in P <= np.quantile(P, 0.15)] 
color_high_O = ['#1f77b4' if top else '#7f7f7f' for top in O >= np.quantile(O, 0.5)]

In [ ]:
import rpy2
### low-level API
import rpy2.rinterface as rinterface
### high-level API 
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

with localconverter(ro.default_converter + pandas2ri.converter):
    vis_df_r = ro.conversion.py2rpy(pd.DataFrame(vis_df))
    vis_label_r = ro.conversion.py2rpy(tissue_ns)
    color_r_membership = ro.conversion.py2rpy(color_membership)
    color_r_high_C2 = ro.conversion.py2rpy(color_high_C2)
    color_r_high_mpg = ro.conversion.py2rpy(color_high_mpg)
    color_r_low_P = ro.conversion.py2rpy(color_low_P)
    color_r_high_O = ro.conversion.py2rpy(color_high_O)
    
### Load into R environment
    ro.globalenv["vis_df_r"] = vis_df_r
    ro.globalenv["vis_label_r"] = vis_label_r
    ro.globalenv["color_r_membership"] = color_r_membership
    ro.globalenv["color_r_high_C2"] = color_r_high_C2
    ro.globalenv["color_r_high_mpg"] = color_r_high_mpg
    ro.globalenv["color_r_low_P"] = color_r_low_P
    ro.globalenv["color_r_high_O"] = color_r_high_O
    ro.r(
        '''
            save(vis_df_r, file = "vis_df.Rdata")  
            save(vis_label_r, file = "vis_label.Rdata")
            save(color_r_membership, file = "color_membership.Rdata")
            save(color_r_high_C2, file = "color_r_high_C2.Rdata")
            save(color_r_high_mpg, file = "color_r_high_mpg.Rdata")
            save(color_r_low_P, file = "color_r_low_P.Rdata")
            save(color_r_high_O, file = "color_r_high_O.Rdata")
        ''' 
        )
print ("Transfer successfully!")

In [ ]:
hub_color_map = sns.color_palette(palette = 'Spectral', as_cmap = True)
hub_score = copy.deepcopy(multiplex_nw_classical_monocyte.multiplex_property['per_layer_hub'])
hub_score.append(multiplex_nw_classical_monocyte.multiplex_property['Jaccard_hub'])
hub_score = np.array(hub_score)
color_hub_per_layer = []
for i in range(hub_score.shape[0]):
    data = hub_score[i,:]
    data_norm = (data - np.min(data))/(np.max(data) - np.min(data))
    _color_rgb = hub_color_map(X = data_norm, alpha = None).tolist()
    _color_layer = [matplotlib.colors.to_hex(item, keep_alpha=False) for item in _color_rgb]
    color_hub_per_layer.append(_color_layer)

with localconverter(ro.default_converter + pandas2ri.converter):
    color_hub_per_layer = ro.conversion.py2rpy(color_hub_per_layer)
    ro.globalenv["color_hub_per_layer"] = color_hub_per_layer
    ro.r(
        '''
            save(color_hub_per_layer, file = "color_hub_per_layer.Rdata")
        ''' 
        )

In [ ]:
# weight_color_map = sns.color_palette(palette = 'Spectral', as_cmap = True)
# weight_score = np.array([np.sum(multiplex_nw_classical_monocyte.adj_Jaccard, axis = 0), \
#  *[np.sum(mat, axis = 0) for mat in multiplex_nw_classical_monocyte.adj_filtered_lst]])
# color_weight_per_layer = []
# for i in range(weight_score.shape[0]):
#     data = weight_score[i,:]
#     data_norm = (data - np.min(data))/(np.max(data) - np.min(data))
#     _color_rgb = weight_color_map(X = data_norm, alpha = None).tolist()
#     _color_layer = [matplotlib.colors.to_hex(item, keep_alpha=False) for item in _color_rgb]
#     color_weight_per_layer.append(_color_layer)

# with localconverter(ro.default_converter + pandas2ri.converter):
#     color_weight_per_layer = ro.conversion.py2rpy(color_weight_per_layer)
#     ro.globalenv["color_weight_per_layer"] = color_weight_per_layer
#     ro.r(
#         '''
#             save(color_weight_per_layer, file = "color_weight_per_layer.Rdata")
#         ''' 
#         )

# num_color_map = sns.color_palette(palette = 'Spectral', as_cmap = True)
# num_score = np.array([np.sum(multiplex_nw_classical_monocyte.adj_Jaccard>0, axis = 0), \
#  *[np.sum(mat>0, axis = 0) for mat in multiplex_nw_classical_monocyte.adj_filtered_lst]])
# color_num_per_layer = []
# for i in range(weight_score.shape[0]):
#     data = num_score[i,:]
#     data_norm = (data - np.min(data))/(np.max(data) - np.min(data))
#     _color_rgb = num_color_map(X = data_norm, alpha = None).tolist()
#     _color_layer = [matplotlib.colors.to_hex(item, keep_alpha=False) for item in _color_rgb]
#     color_num_per_layer.append(_color_layer)

# with localconverter(ro.default_converter + pandas2ri.converter):
#     color_num_per_layer = ro.conversion.py2rpy(color_num_per_layer)
#     ro.globalenv["color_num_per_layer"] = color_num_per_layer
#     ro.r(
#         '''
#             save(color_num_per_layer, file = "color_num_per_layer.Rdata")
#         ''' 
#         )